In [1]:
import sys
sys.version

import re
#from konlpy.tag import Okt          
from konlpy.tag import Komoran
from collections import Counter
import pandas as pd 

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

## 한글 폰트 설정

In [2]:
font_path ="C:/Windows/fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font',family=font_name)

In [3]:
train = pd.read_csv('_data/train.csv', index_col=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  5000 non-null   object
 1   label     5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [4]:
test = pd.read_csv('_data/test.csv', index_col=0)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  5000 non-null   object
dtypes: object(1)
memory usage: 78.1+ KB


# 형태소 및 단어 분리
* 영어만으로 작성된 리뷰도 있기 때문에 문자열을 제외한 나머지를 공백으로 치환 (re.sub(r'[^\w]+', ' ', x)

In [74]:
nlp = Komoran()
def preprocessing(df):
    df['document'] = df.document.apply(lambda x : re.sub(r'[^\w]+', ' ', x))
    df['morphs'] = df.document.apply(lambda x : nlp.morphs(x))
    df['nouns'] = df.document.apply(lambda x : nlp.nouns(x))
    
    for i, strings in enumerate(df.morphs):
        after = strings
        for j, s in enumerate(strings):
            if len(s)<2:
                after.pop(j)
        if len(strings)!=len(after):
            df.ioc[i, 'morphs'] = after
    
    for i, strings in enumerate(df.nouns):
        after = strings
        for j, s in enumerate(strings):
            if len(s)<2:
                after.pop(j)
        if len(strings)!=len(after):
            df.ioc[i, 'nouns'] = after
    
    return df

In [75]:
train = preprocessing(train)
train.head()

,document,label,morphs,nouns
id,,,,
1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0,영상 이나 음악 이 이쁘 다 하 아도 미화 시키 불륜 일 뿐,"[영상, 음악, 미화, 불륜, 일]"
2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1,히치콕 이 이 영화 를 보 았 다면 분명 박수 를 치 었 을 듯,"[히치콕, 영화, 박수, 듯]"
3,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1,괜찮 은 음악 영화 가 또 나오 았 군요 따뜻 하 겨울 이 되 것 같 아요,"[음악, 영화, 겨울, 것]"
4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0,아무래도 년대 지난 작품 이 라 지금 보기 는 너무 유치 하 다,"[년대, 지난, 작품, 지금, 보기, 유치]"
5,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,0,지금 까지 의 영화 들 이 그렇 었 듯 이 영화 역시 일본 에 대하 미화 는 여전 하 다,"[지금, 영화, 영화, 일본, 미화]"


In [64]:
test = preprocessing(test)
test.head()

,document,morphs,nouns
id,,,
1,시간 때우기 좋은 영화 지루함,"[시간, 때우, 좋, 영화, 지루함]","[시간, 영화, 지루함]"
2,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,"[훈훈, ㄴ, 정이, 느끼, 지, 영화, 가족, 끼리, 드라마, 듯이, 면]","[정이, 영화, 가족, 드라마]"
3,Childhood fantasy,"[Childhood, fantasy]",[]
4,멋있는 영화입니다 잊을 수 없는,"[멋있, 영화, ㅂ니다, 을, 없]",[영화]
5,너무 감동적이네요 펑펑 울었습니다,"[너무, 감동, 이, 네요, 펑펑, 었, 습니다]",[감동]


In [65]:
positive = train[train.label==1]
positive.tail()

,document,label,morphs,nouns
id,,,,
4989,저 덕심 쥐뿔도 없지만 정말 재미있었어요,1,"[덕, 쥐뿔, 없, 지만, 정말, 재미있, 어요]","[심, 쥐뿔]"
4990,진짜재미있었다 조금지그러웠지만역시이토준지씨이다,1,"[진짜, 재미있, 다, 조금지그러웠지만역시이토준지씨이다]",[진짜]
4993,아 오랜 기다림끝에 드디어 보게 되네요 최고입니다,1,"[오랜, 기다리, 끝, 드디어, 게, 네요, 최고, ㅂ니다]",[최고]
4994,고난도의 심리 묘사가 표현된 최고의 영화,1,"[고난, 의, 심리, 묘사, 표현, ㄴ, 최고, 영화]","[고난, 심리, 묘사, 표현, 최고, 영화]"
4999,이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가,1,"[이런, 평점, 테러, 네이버, 에서, 막, 아야, 는, 아니, ㄴ가]","[평점, 테러, 네이버]"


In [66]:
negative = train[train.label==0]
negative.tail()

,document,label,morphs,nouns
id,,,,
4995,도대체 누가 안소니짐머라는거지,0,"[도대체, 누구, 안소니, 짐머, 라는, 거지]","[안소니, 짐머, 거지]"
4996,좋은 배우들로 3류영화를 찍은 안타까운 영화,0,"[은, 배우, 로, 류, 영화, 찍, 안타깝, 영화]","[배우, 영화, 영화]"
4997,진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화,0,"[진짜, 드럽, 재미없, 에드워드 호퍼, 그림, 배경, 고, 보, ㄹ게, 아니, 영화]","[에드워드 호퍼, 그림, 배경, 영화]"
4998,가장 실망스러운 영화 지금까지 본영화중,0,"[가장, 실망, 스럽, 영화, 지금, 까지, ㄴ, 영화]","[실망, 영화, 지금, 영화]"
5000,여주인공이 전작에서는 귀여웠는데 여기선 완전 망했네 실망이다,0,"[여주인공, 전작, 에서, 귀엽, 는데, 여기, ㄴ, 완전, 망하, 네, 실망, 다]","[여주인공, 전작, 완전, 실망]"


In [43]:
posi_train = df2str(train['nouns'])
posi_train

5000


'영상 음악 미화 불륜'

In [25]:
c =Counter(posi_train)

In [26]:
c.most_common(10)

[(' ', 27944),
 ('하', 2641),
 ('화', 1835),
 ('영', 1813),
 ('보', 1464),
 ('지', 1346),
 ('도', 1191),
 ('이', 1027),
 ('기', 1009),
 ('의', 937)]

In [11]:
data['len'] = data.document.apply(lambda x : len(x))

NameError: name 'data' is not defined

In [ ]:
data['split_len'] = data.document.apply(lambda x : len(x.split()))

In [ ]:
data['len/split'] = data['len']/data['split_len']

In [ ]:
data[data['len/split']>6]